### Selecting & Displaying the Dataframe To be use in this Sentiment Analysis 

In [ ]:
df = spark.sql("SELECT * FROM OpenWealth_Lakehouse.table_latest_news")
display(df)


### Importing the pre-built Intelligent model(AnalyzeText)

In [ ]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

### Configuring the Input and Output Columns to the model

In [ ]:
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

### Applying the model to our dataframe.
Note: Here we are using transform() function to give our dataframe as input to the model.

In [ ]:
result=model.transform(df)

### Displaying the output produced by the model

In [ ]:
display(result)

### Extracting the only a sentiment response from the output produced by the model 

In [ ]:
from pyspark.sql.functions import col
sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

### Displaying the dataframe after the extraction of response from the existing output by the model

In [ ]:
display(sentiment_df)

### Removing the two unused columns (error,response) from the existing dataframe

In [ ]:
sentiment_df_final = sentiment_df.drop("error","response")

### Displaying the final dataframe 

In [ ]:
display(sentiment_df_final)

In [17]:
from pyspark.sql.functions import col,to_date
sentiment_df_final=sentiment_df_final.withColumn("PublishedAt",to_date(col("PublishedAt"),"dd-MMM-yyyy"))

StatementMeta(, 9e460d79-6f79-4330-8d06-a732b9176966, 19, Finished, Available, Finished)

### Converting the final dataframe to delta format & storing this new table in the lakehouse.
### Note: Here we are using incremental load technique type 1 (Overwritten & No history)

In [ ]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'OpenWealth_Lakehouse.table_Sentimental_Analysis'
    
    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:
    print("Table Already Exists")
    sentiment_df_final.createOrReplaceTempView("vw_df_Sentimental_final")
    spark.sql(f"""
    MERGE INTO {table_name} target_table
    USING vw_df_Sentimental_final source_view
    ON source_view.url = target_table.url
    WHEN MATCHED THEN
    UPDATE SET
        author = source_view.author,
        content = source_view.content,
        description = source_view.description,
        publishedAt = source_view.publishedAt,
        source = source_view.source,
        title = source_view.title,
        urlToImage = source_view.urlToImage
    WHEN NOT MATCHED THEN
    INSERT *
    """)
